In [1]:
import requests, json

In [2]:
LANGUAGE_ENDPOINT = "https://proj-rnd-uae-et-language-001.cognitiveservices.azure.com/"
LANGUAGE_KEY = "0b938542471248fda3aab56075a251c1"

words_sentiment_mapping_flag = True
target_words_flag = True

def send_request(dialogue):
    data= {
                "kind": "SentimentAnalysis",
                "parameters": {
                    "modelVersion": "latest",
                    "opinionMining": "True"
                },
                "analysisInput":{
                    "documents":[
                        {
                            "id":"1",
                            "language":"en",
                            "text": dialogue
                        }
                    ]
                }
            }

    url = f"{LANGUAGE_ENDPOINT}/language/:analyze-text?api-version=2023-04-15-preview"

    headers = {
        "Content-Type": "application/json",
        "Ocp-Apim-Subscription-Key": LANGUAGE_KEY
    }

    sentiment_response = requests.post(url= url, json=data, headers= headers)
    return sentiment_response

In [3]:
# sentiment_response.status_code

In [4]:
def get_sentiment(sentiment_result):
    overall_sentiment = sentiment_result["results"]["documents"][0]["sentiment"]
    print(f"overall sentiment : ", overall_sentiment)
    dic1 = sentiment_result["results"]["documents"][0]["confidenceScores"]
    max_sentiment = max(dic1, key=dic1.get)
    print(f"sentiment : {max_sentiment}")
    return overall_sentiment, max_sentiment

In [5]:
def get_words_sentiment_mapping(sentiment_result):
    words_sentiment_mapping = []
    for item in sentiment_result["results"]["documents"][0]["sentences"]:
        opinions  = item["targets"]
        # print(len(opinions)) #depends on how many text-sentiment is predicted
        # print(f"{opinions}\n\n")
        for opinion in opinions:
            sentiment = opinion["sentiment"]
            word = opinion["text"]
            pair = (word, sentiment)
            words_sentiment_mapping.append(pair)
    
    return words_sentiment_mapping

In [6]:
def get_sentiment_analysis(sentiment_result):
    sentiment = get_sentiment(sentiment_result)[0]
    result = {"sentiment": sentiment}
    if words_sentiment_mapping_flag:
        mapping = get_words_sentiment_mapping(sentiment_result)
        result["words_sentiment_mapping"] = mapping
        if mapping and target_words_flag:
            target_words = [item[0] for item in mapping]
            result["target_words"] = target_words
    
    return result

In [7]:
def sentiment_helper():
    sentiment_response = send_request(dialogue="")
    sentiment_result = json.loads(sentiment_response.text)

    if sentiment_result["results"]["errors"]:
        error_msg =  f"status code : {sentiment_response.status_code}. Response : {sentiment_response.text}"
        error_msg = "".join(sentiment_result["results"]["errors"])
        result = {"status": "fail", "error": error_msg}
    else:
        result = get_sentiment_analysis(sentiment_result)
        result["status"] = "success"

In [9]:
sentiment_helper()

KeyError: 'results'

In [31]:
sentiment_result["results"]["documents"]

[{'id': '1',
  'sentiment': 'mixed',
  'confidenceScores': {'positive': 0.43, 'neutral': 0.04, 'negative': 0.53},
  'sentences': [{'sentiment': 'negative',
    'confidenceScores': {'positive': 0.0, 'neutral': 0.01, 'negative': 0.99},
    'offset': 0,
    'length': 40,
    'text': 'The food and service were unacceptable. ',
    'targets': [{'sentiment': 'negative',
      'confidenceScores': {'positive': 0.01, 'negative': 0.99},
      'offset': 4,
      'length': 4,
      'text': 'food',
      'relations': [{'relationType': 'assessment',
        'ref': '#/documents/0/sentences/0/assessments/0'}]},
     {'sentiment': 'negative',
      'confidenceScores': {'positive': 0.01, 'negative': 0.99},
      'offset': 13,
      'length': 7,
      'text': 'service',
      'relations': [{'relationType': 'assessment',
        'ref': '#/documents/0/sentences/0/assessments/0'}]}],
    'assessments': [{'sentiment': 'negative',
      'confidenceScores': {'positive': 0.01, 'negative': 0.99},
      'offset':